# Shop incentory analysis and ETL

The purpose of this notebook is to convert data provided by the client to the expected format, so they are loadable into SQL db. The structure of this file divides it into classic 3 sections:

1. Extract -> loading the data, clearing any issues
2. Transform -> reshape the data
3. Load -> check the quality and save the results

## 1. Extract

In [ ]:
import pandas as pd

In [ ]:
!ls inputs/

There are 3 files delivered: inventory, prices and meta. First, a look into inventory.

## 1.1. Inventory

In [ ]:
!head inputs/coding_challenge_inventory.csv

In [ ]:
inventory = pd.read_csv("inputs/coding_challenge_inventory.csv", index_col=False)
inventory

In [ ]:
inventory.dtypes

In [ ]:
inventory.describe()

It looks like a properly formatted CSV file, first two columns are described as integers. What can worry a bit here is the negative inventory level for some items, let's have a closer look:

In [ ]:
inventory[inventory["inventory_level"] < 0][["item_number", "inventory_level"]]

Entries with `-1` and `-2` look like a counting issue (I guess it can happen in real life scenarios in shops), but `-2259` and `-12738` look quite suspicious (considering the max for positive numbers is `1400`). Either there was some issue on generating these, of something is really odd with these products. We're going to import these, but it's worthy to ask the client for explanation.

Also, the final format requires report timestamp as one field, let's check if all seems fine here:

In [ ]:
inventory["report_timestamp"] = pd.to_datetime(
    inventory["report_date"] + " " + inventory["report_time"]
)
inventory[inventory["report_timestamp"].isnull()]

All entries are properly converted, no need to worry and we can move on to the next file

## 1.2. Meta

In [ ]:
!head inputs/coding_challenge_meta.csv

In [ ]:
try:
    pd.read_csv("inputs/coding_challenge_meta.csv", index_col=False)
except pd.errors.ParserError as e:
    print(e)

Parser fails in the very beginning, so it looks the `.csv` extension here doesn't really mean it's a valid CSV file... Let's have a look on reported line 33:

In [ ]:
!cat inputs/coding_challenge_meta.csv | sed -n '1p; 33p'

First fields look propely formatted:

* product_upc: 000000000001030054325
* item_number: 10061658
* item_description: Eloquent Modest Mestorf
* case_upc: null
* case_pack: 12.0
* department: 1

But the last field, `supplier`, looks like not propetly quoted string with comma inside (as name of the company "Sth, Inc" is quite common form). We're going to fix it by parsing supplier having in mind this issue.

In [ ]:
def line_fixer(arr):
    number_in_case_pack_column = arr[4].replace(".", "").isnumeric()
    number_in_department_column = arr[5].isnumeric()
    if number_in_case_pack_column and number_in_department_column:
        arr[6] = ",".join(arr[6:])
    else:
        raise ValueError("Unhandled issue")
    arr = arr[:7]
    return arr


meta = pd.read_csv(
    "inputs/coding_challenge_meta.csv", on_bad_lines=line_fixer, engine="python"
)

In [ ]:
meta.dtypes

In [ ]:
meta.describe()

Numeric columns look good - `product_upc` and `item_number` is always there, `case_upc` is optional (only on 167 rows) and case_pack is missing on a 6 records.

But, there is something wrong with `department` column - looking the the file in text format indicates that this is an integer (with all values `1` in the first 9 records), but pandas sees this as `object` (so probably a string). Let's see the set of values:

In [ ]:
meta["department"].unique()

In [ ]:
non_numeric_department_values = [
    dept
    for dept in meta["department"].unique()
    if isinstance(dept, str) and not dept.isnumeric()
]
rows_with_wrong_department = meta[
    meta["department"].isin(non_numeric_department_values)
]

In [ ]:
try:
    meta.loc[[23, 24]]
except Exception:
    pass

In [ ]:
try:
    meta.loc[[82, 83]]
except Exception:
    pass

It looks like a nother issue in the CSV file with department being ommited from the consecutive rows for the same `item_number`... Let's double check if this is always the case:

In [ ]:
for row in rows_with_wrong_department.iterrows():
    previous_row_supplier = meta.loc[row[0] - 1, "supplier"]
    invalid_row_department = row[1].department
    previous_row_item_number = meta.loc[row[0] - 1, "item_number"]
    invalid_row_item_number = row[1].item_number
    assert previous_row_supplier == invalid_row_department
    assert previous_row_item_number == invalid_row_item_number

Yes, it's always the case. Thankfully, we need supplier id only on `item_number` level (not `product_upc` level), so we can basically clear the invalid department values. Or - even easier - since the deparment doesn't need to be imported into final SQL db, we can drop the entire column.

In [ ]:
meta = meta.drop("department", axis=1)

In [ ]:
meta

## 1.3. Prices

In [ ]:
prices = pd.read_csv("inputs/coding_challenge_prices.csv", index_col=False)

In [ ]:
prices

In [ ]:
prices.dtypes

In [ ]:
prices.describe()

Third csv looks really good - nothing suspicious there, just a 2-column standard CSV file.

# 2. Transform

First, let's find the products in `meta` that are not real products but variants.

# 2.1. Handling duplicates UPCs

The final data require UPC column with unique constraint. According to the documentation:

> HINT: If a `upc` has multiple `item_number` values in the product metadata file, then that
means the product has multiple variants.

In [ ]:
meta["duplicated_upcs"] = meta.groupby("product_upc").item_number.transform("nunique")
duplicated_upcs = meta[meta["product_upc"].duplicated()]["product_upc"].to_list()
meta[meta["duplicated_upcs"] > 1].sort_values("product_upc")

So, we have 2 cases with variants. Unfortunately, their descriptions and item numbers do not match, this indicates probable issue with `upc` data entry. What is more, it looks prices also contain duplicates for these `upc` codes:

In [ ]:
prices[prices["product_upc"].isin(duplicated_upcs)]

In the normal curcuimstances, these issues with unique identifiers should be consulted with team responsible for the data delivery, but in our case, for simplicity and to make sure we can make `upc` a unique column, I will assume the first row for every product is the proper indicator of `item_number` and `item_description`, and the second row is a variant.

In [ ]:
meta = meta[~meta["product_upc"].duplicated()].copy()
prices = prices[~prices["product_upc"].duplicated()].copy()

Eventually, `product_upc` should be unique across `meta` and `prices`:

In [ ]:
meta[meta["product_upc"].duplicated()]

In [ ]:
prices[prices["product_upc"].duplicated()]

Let's check the current row indices for these:

In [ ]:
meta[meta["product_upc"].isin(duplicated_upcs)]

And, we can build variants alternates now:

In [ ]:
variants_duplicated_upcs = (
    meta[meta["product_upc"].isin(duplicated_upcs)]
    .reset_index()[["index", "product_upc"]]
    .rename(columns={"index": "product_id", "product_upc": "upc"})
)
variants_duplicated_upcs["alternate_type"] = "variant"
variants_duplicated_upcs

## 2.2. Handling variants of products

Then, we need to handle variants, so rows when one item has multiple UPCs. First, lets ensure that all the items have the same description:

In [ ]:
meta["unique_upcs"] = meta.groupby("item_number").product_upc.transform("nunique")
meta["unique_names"] = meta.groupby("item_number").item_description.transform("nunique")
meta[(meta["unique_upcs"] > 1) & (meta["unique_names"] != 1)]

Thankfully, all the item names are consistent across variants. So we just create a set of variants by using first row as primary product, and a second one as a duplicate:

In [ ]:
items_having_variants = meta[meta["item_number"].duplicated()]["item_number"].to_list()
all_products_with_wariants = meta[meta["item_number"].isin(items_having_variants)]
main_products = all_products_with_wariants[
    ~all_products_with_wariants["item_number"].duplicated()
]
main_products_mapping = main_products.reset_index()[["item_number", "index"]].rename(
    columns={"index": "product_id"}
)

In [ ]:
variants = (
    meta[meta["item_number"].duplicated()]
    .merge(main_products_mapping, on="item_number")[["product_id", "product_upc"]]
    .rename(columns={"product_upc": "upc"})
)
variants["alternate_type"] = "variant"
variants

In [ ]:
meta = meta[~meta["item_number"].duplicated()].copy()

## 2.3. Handling case alternates

Finally, creating a set of case alternates seems quite simple:

In [ ]:
cases = (
    meta[meta["case_upc"].notnull()]
    .reset_index()[["index", "case_upc", "case_pack"]]
    .rename(columns={"index": "product_id", "case_upc": "upc"})
)
cases["alternate_type"] = "case"
cases

## 2.4. Creating alternates

Eventually, we combine all the sets:

In [ ]:
product_alternates = pd.concat([variants_duplicated_upcs, variants, cases])
product_alternates["upc"] = (
    product_alternates["upc"].astype("int64").astype("string").str.zfill(14)
)
product_alternates = product_alternates.reset_index(drop=True)
product_alternates

## 2.5. Creating products

In `products` table we need products with their price and inventory. Therefore, first, lets join `meta` and `prices`. We know already that there are no duplicates of join key between these, therefore, the result of left join should contain as many rows as `meta` table

In [ ]:
meta = meta.reset_index()
len(meta)

In [ ]:
meta_with_prices = meta.merge(prices, on="product_upc", how="left")
meta_with_prices

Before joining inventory, let's check for duplicates there:

In [ ]:
inventory[inventory["item_number"].duplicated()]

In [ ]:
meta_with_prices

In [ ]:
products = meta_with_prices.merge(inventory, on="item_number", how="left")
products = products.set_index("index")
products = products[
    [
        "product_upc",
        "item_description",
        "item_number",
        "price",
        "supplier",
        "inventory_level",
        "report_timestamp",
    ]
].rename(
    columns={
        "product_upc": "upc",
        "item_description": "name",
        "report_timestamp": "inventory_updated_at",
    }
)
products["upc"] = products["upc"].astype("int64").astype("string").str.zfill(14)
products

# 3. Load

We have all the tables ready, but before saving them into DB, let's make sure they pass the simple data quality checks.

In [ ]:
# is upc unique?
assert not products["upc"].duplicated().any(), "Duplicates on UPC"

# do we have all products?
meta = pd.read_csv(
    "inputs/coding_challenge_meta.csv", on_bad_lines=line_fixer, engine="python"
)
assert len(products) == len(meta) - len(duplicated_upcs) - len(
    variants
), "Products missing"

# can we consider product_id in alternates a foreign key?
assert len(
    products.reset_index().merge(
        product_alternates, left_on="index", right_on="product_id"
    )
) == len(product_alternates), "Foreign key issue"

All looks clean, it's time to save those as CSVs, so they can be loaded into DB

In [ ]:
products.to_csv("outputs/products.csv", index_label="product_id")
product_alternates.to_csv(
    "outputs/product_alternates.csv", index_label="product_alternate_id"
)